In [ ]:
#! /usr/bin/env python3

import os
import h5py
import pickle
import numpy as np
from sgptools.utils.gpflow import *
from utils import CustomStandardScaler, StandardScaler
from sgptools.utils.misc import ploygon2candidats
from sgptools.utils.metrics import get_rmse, get_distance
from sgptools.kernels.attentive_kernel import AttentiveKernel
from sgptools.kernels.neural_kernel import NeuralSpectralKernel
from gpflow.models.util import data_input_to_tensor
import matplotlib.pyplot as plt
from matplotlib import cm
from skimage.metrics import structural_similarity as ssim

tf.random.set_seed(2024)
np.random.seed(2024)

In [ ]:
# Lawn mower path data
fname = os.path.join('../launch/data/lawn_mower/mission-log.hdf5')
with h5py.File(fname, "r") as f:
    fence_vertices = f["fence_vertices"][:].astype(float)
    X = f["X"][:].astype(float)
    y = f["y"][:].astype(float)

# Normalize the candidates
X_candidates = ploygon2candidats(fence_vertices, 
                                 num_samples=50000)

X_scaler = CustomStandardScaler()
y_scaler = StandardScaler()

X_scaler.fit(X_candidates)
X_scaler.scale_ *= 0.35
X_candidates = X_scaler.transform(X_candidates)    

X = X_scaler.transform(X)
y = y_scaler.fit_transform(y)

print(X.shape, y.shape)

In [ ]:
# Train SGP
kernel_name = 'Attentive'
if kernel_name == 'RBF':
    kernel = gpflow.kernels.RBF(lengthscales=0.1, 
                                variance=0.5)
elif kernel_name == 'Attentive':
    kernel = AttentiveKernel(np.linspace(0.1, 2.5, 10), 
                             dim_hidden=10)
elif kernel_name == 'Neural':
    kernel = NeuralSpectralKernel(input_dim=2, 
                                  Q=5, 
                                  hidden_sizes=[4, 4])

In [ ]:
# Train GP only if pretrained weights are unavailable
fname = os.path.join('../launch/data/lawn_mower', f"{kernel_name}Params.pkl")
with open(fname, 'rb') as handle:
    params = pickle.load(handle)
max_steps = 0

_, _, _, gpr_gt = get_model_params(X, y,
                                   kernel=kernel,
                                   return_gp=True,
                                   train_inducing_pts=True,
                                   max_steps=max_steps)

gpflow.utilities.multiple_assign(gpr_gt.kernel, params['kernel'])
gpflow.utilities.multiple_assign(gpr_gt.likelihood, params['likelihood'])

In [ ]:
# Generate X_test and y_test data
X_ = np.linspace(-5, 5, 100)
Y_ = np.linspace(-5, 5, 100)
X_, Y_ = np.meshgrid(X_, Y_)
X_test = np.vstack([X_.ravel(), Y_.ravel()]).T

y_test = gpr_gt.predict_f(X_test)[0].numpy()
y_test = -np.clip(y_test, y.min(), y.max())

In [ ]:
plt.figure()
ax = plt.gca()
ax.set_aspect('equal')
plt.scatter(X_test[:, 0], X_test[:, 1], 
            c=y_test, cmap=cm.terrain)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('SRTM Elevation Data')
plt.savefig('GT.pdf', bbox_inches='tight')
plt.show()

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')
surf = ax.plot_surface(X_test[:, 0].reshape(100, 100), 
                       X_test[:, 1].reshape(100, 100), 
                       y_test.reshape(100, 100), 
                       cmap=cm.terrain)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Elevation')
ax.view_init(elev=45, azim=-145)
ax.set_box_aspect(None, zoom=0.85)
plt.savefig('GT_3d.pdf', bbox_inches='tight')
plt.show()

In [ ]:
_, _, _, gpr_gt = get_model_params(X[:10], y[:10],
                                   kernel=kernel,
                                   return_gp=True,
                                   max_steps=0)


In [ ]:
# Train SGP
for kernel_name in ['RBF', 'Attentive', 'Neural']:
    if kernel_name == 'RBF':
        kernel = gpflow.kernels.RBF(lengthscales=0.1, 
                                    variance=0.5)
    elif kernel_name == 'Attentive':
        kernel = AttentiveKernel(np.linspace(0.1, 2.5, 10), 
                                dim_hidden=10)
    elif kernel_name == 'Neural':
        kernel = NeuralSpectralKernel(input_dim=2, 
                                    Q=5, 
                                    hidden_sizes=[4, 4])

    # Train GP only if pretrained weights are unavailable
    fname = os.path.join('../launch/data/lawn_mower', f"{kernel_name}Params.pkl")
    with open(fname, 'rb') as handle:
        params = pickle.load(handle)

    gpr_gt.kernel = kernel
    gpflow.utilities.multiple_assign(gpr_gt.kernel, params['kernel'])
    gpflow.utilities.multiple_assign(gpr_gt.likelihood, params['likelihood'])

    fname = os.path.join(f'../launch/data/{kernel_name}/mission-log.hdf5')
    with h5py.File(fname, "r") as f:
        fence_vertices = f["fence_vertices"][:].astype(float)
        X_train = f["X"][:].astype(float)
        y_train = f["y"][:].astype(float)

    X_train = X_scaler.transform(X_train)    
    y_train = y_scaler.transform(y_train)
    gpr_gt.data = data_input_to_tensor((X_train, y_train))

    print(X_train.shape, y_train.shape)

    y_pred = gpr_gt.predict_f(X_test)[0].numpy()
    y_pred = -np.clip(y_pred, y.min(), y.max())
    ssim_measure = ssim(y_pred.reshape(100, 100), 
                        y_test.reshape(100, 100), 
                        data_range=y_test.max() - y_test.min())
    dist = get_distance(X_train)

    plt.figure()
    ax = plt.gca()
    ax.set_aspect('equal')
    plt.scatter(X_test[:, 0], X_test[:, 1], 
                c=y_pred, cmap=cm.terrain)
    plt.plot(X_train[:, 0], X_train[:, 1], '--', c='r')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(f'RMSE: {get_rmse(y_pred, y_test):.2f} - SSIM: {ssim_measure:.2f} - Distance: {dist:.2f}')
    plt.savefig(f'{kernel_name}Kernel.pdf', bbox_inches='tight')
    plt.show()

    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(projection='3d')
    surf = ax.plot_surface(X_test[:, 0].reshape(100, 100), 
                        X_test[:, 1].reshape(100, 100), 
                        y_pred.reshape(100, 100), 
                        cmap=cm.terrain)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Elevation')
    ax.view_init(elev=45, azim=-145)
    ax.set_box_aspect(None, zoom=0.90)
    plt.savefig(f'{kernel_name}Kernel_3d.pdf', bbox_inches='tight')
    plt.show()